In [1]:
import pandas as pd
import glob

In [ ]:
count = 0
# Read all CSV files in the specified directory
for folder in ["2010-2011", "2011-2012"]:
    for file in glob.glob(f"./processed_data/{folder}/*.csv"):
        df = pd.read_csv(file)
        if count == 0:
            df_agg = df
        else:
            df_agg = pd.concat([df_agg, df])
        
        count += 1
df_agg.to_csv("./model_input_data/2010-2012.csv", index=False)

In [ ]:
count = 0
# Read all CSV files in the specified directory
for file in glob.glob("./processed_data/2012-2013/*.csv"):
    df = pd.read_csv(file)
    if count == 0:
        df_agg = df
    else:
        df_agg = pd.concat([df_agg, df])
    
    count += 1
df_agg.to_csv("./model_input_data/2012-2013.csv", index=False)

In [30]:
dfs = []
# Read all CSV files in the specified directory
for folder in ["2010-2011", "2011-2012"]:
    for file in glob.glob(f"./processed_data/{folder}/*.csv"):
        df = pd.read_csv(file)
        dfs.append(df)

df_agg = pd.concat(dfs, ignore_index=True)
df_agg = df_agg.sort_values(by=["Customer", "datetime"])

# Restrict time from 5am to 8pm
df_agg["datetime"] = pd.to_datetime(df_agg["datetime"])
df_agg = df_agg[(df_agg["datetime"].dt.hour >= 5) & (df_agg["datetime"].dt.hour <= 20)]

# Keep relevant columns
df_agg = df_agg[["Customer", "datetime", "GG", "NL"]]

df_agg.to_csv("./model_input_data/2010-2012_combined.csv", index=False)

In [31]:
dfs = []
# Read all CSV files in the specified directory
for file in glob.glob("./processed_data/2012-2013/*.csv"):
        df = pd.read_csv(file)
        dfs.append(df)

df_agg = pd.concat(dfs, ignore_index=True)
df_agg = df_agg.sort_values(by=["Customer", "datetime"])

# Restrict time from 5am to 8pm
df_agg["datetime"] = pd.to_datetime(df_agg["datetime"])
df_agg = df_agg[(df_agg["datetime"].dt.hour >= 5) & (df_agg["datetime"].dt.hour <= 20)]

# Keep relevant columns
df_agg = df_agg[["Customer", "datetime", "GG", "NL"]]

df_agg.to_csv("./model_input_data/2012-2013_combined.csv", index=False)

In [2]:
for i in range(1, 13):
    season_number = i % 12 // 3 + 1
    season_name = "summer" if i in [12, 1, 2] else "autumn" if i in [3, 4, 5] else "winter" if i in [6, 7, 8] else "spring"
    print(f"{i} {season_name} {season_number}")

1 summer 1
2 summer 1
3 autumn 2
4 autumn 2
5 autumn 2
6 winter 3
7 winter 3
8 winter 3
9 spring 4
10 spring 4
11 spring 4
12 summer 1


In [33]:
# Add feature columns
for file in glob.glob("./model_input_data/*.csv"):
    df = pd.read_csv(file)
    print(f"Processing file: {file}")

    df["datetime"] = pd.to_datetime(df["datetime"])  # Ensure datetime is in correct format
    
    df["NL_t-24"] = df["NL"].shift(48)  # Shift by 48 half-hours (1 day)
    df["NL_t-24*7"] = df["NL"].shift(48*7)  # Shift by 336 half-hours (7 days)
    df["NL_t-24*30"] = df["NL"].shift(48*30)  # Shift by 1440 half-hours (30 days)
    df["Month"] = df["datetime"].dt.month # Extract month from datetime
    df["Season"] = (df["Month"] % 12) // 3 + 1 # Convert month to season (1-4)
    df["is_weekend"] = (df["datetime"].dt.weekday >= 5).astype(int)  # 1 if Saturday or Sunday
    
    # Save the updated DataFrame back to CSV
    df.to_csv(file, index=False)

Processing file: ./model_input_data/2012-2013_combined.csv
Processing file: ./model_input_data/2010-2012_combined.csv


In [9]:
for folder in ["2010-2011", "2011-2012", "2012-2013"]:
    for file in glob.glob(f"./processed_data/{folder}/*.csv"):
        # Print unique dates in the 'datetime' column
        df = pd.read_csv(file)
        df["datetime"] = pd.to_datetime(df["datetime"])
        unique_dates = df["datetime"].dt.date.unique()

        # Print files with missing dates
        if len(unique_dates) < 365:
            print(f"File {file} has missing dates: {len(unique_dates)} unique dates found.")

        #print(f"Unique dates in {file}: {unique_dates}")
        #print(f"Number of unique dates in {file}: {len(unique_dates)}")

File ./processed_data/2012-2013/NL_2012-2013_2.csv has missing dates: 284 unique dates found.


In [10]:
dfs = []
# Read all CSV files in the specified directory
for folder in ["2010-2011", "2011-2012", "2012-2013"]:
    for file in glob.glob(f"./processed_data/{folder}/*.csv"):
        df = pd.read_csv(file)
        dfs.append(df)

df_agg = pd.concat(dfs, ignore_index=True)
df_agg = df_agg.sort_values(by=["Customer", "datetime"])
df_agg.to_csv("./model_input_data/2010-2013_combined.csv", index=False)

# Keep relevant columns
df_agg = df_agg[["Customer", "datetime", "GG", "NL"]]

#df_agg["datetime"] = pd.to_datetime(df_agg["datetime"])  # Ensure datetime is in correct format

# Add feature columns
#df_agg["NL_t-24"] = df_agg.groupby("Customer")["NL"].shift(48)  # Shift by 48 half-hours (1 day)
#df_agg["NL_t-24*7"] = df_agg.groupby("Customer")["NL"].shift(48*7)  # Shift by 336 half-hours (7 days)
#df_agg["NL_t-24*30"] = df_agg.groupby("Customer")["NL"].shift(48*30)  # Shift by 1440 half-hours (30 days)

#df_agg["Month"] = df_agg["datetime"].dt.month  # Extract month from datetime
#df_agg["Season"] = (df_agg["Month"] % 12) // 3 + 1  # Convert month to season (1-4)
#df_agg["is_weekend"] = (df_agg["datetime"].dt.weekday >= 5).astype(int)  # 1 if Saturday or Sunday

# Restrict time from 5am to 8pm
#df_agg = df_agg[(df_agg["datetime"].dt.hour >= 5) & (df_agg["datetime"].dt.hour <= 20)]

#df_train = df_agg[df_agg["datetime"] < "2012-07-01"]
#df_test = df_agg[df_agg["datetime"] >= "2012-07-01"]
#df_train.to_csv("./model_input_data/2010-2012.csv", index=False)
#df_test.to_csv("./model_input_data/2012-2013.csv", index=False)
df_agg.to_csv("./model_input_data/2010-2013_processed.csv", index=False)

In [11]:
#df_train = pd.read_csv("./model_input_data/2010-2012.csv")
#df_test = pd.read_csv("./model_input_data/2012-2013.csv")

In [12]:
#df_train.head()

In [13]:
#df_test.head()

In [14]:
df = pd.read_csv("./model_input_data/2010-2013_processed.csv")
df.head()

,Customer,datetime,GG,NL
0,1,2010-07-01 00:00:00,0.0,1.200
1,1,2010-07-01 00:30:00,0.0,1.553
2,1,2010-07-01 01:00:00,0.0,1.715
3,1,2010-07-01 01:30:00,0.0,1.339
4,1,2010-07-01 02:00:00,0.0,0.865


In [15]:
# Split data into 4 communities of 75 customers
communities = {
    "community_1": range(1, 76),
    "community_2": range(76, 151),
    "community_3": range(151, 226),
    "community_4": range(226, 301),
}

for community, cid in communities.items():
    # Select customers in the current community
    print(f"Processing {community} with customer IDs: {cid}")

    df_community = df[df["Customer"].isin(cid)]
    df_community.to_csv(f"./model_input_data/{community}.csv", index=False)

    df_community = df_community.groupby("datetime")[["Customer", "GG", "NL"]].sum().reset_index()
    df_community.to_csv(f"./model_input_data/{community}_agg.csv", index=False)
    df_community = df_community.drop(columns=["Customer"])

    df_community["datetime"] = pd.to_datetime(df_community["datetime"])  # Ensure datetime is in correct format

    # Add feature columns
    df_community["NL_t-24"] = df_community["NL"].shift(48)  # Shift by 48 half-hours (1 day)
    df_community["NL_t-24*7"] = df_community["NL"].shift(48*7)  # Shift by 336 half-hours (7 days)
    df_community["NL_t-24*30"] = df_community["NL"].shift(48*30)  # Shift by 1440 half-hours (30 days)

    df_community["Month"] = df_community["datetime"].dt.month  # Extract month from datetime
    df_community["Season"] = (df_community["Month"] % 12) // 3 + 1  # Convert month to season (1-4)
    df_community["is_weekend"] = (df_community["datetime"].dt.weekday >= 5).astype(int)  # 1 if Saturday or Sunday

    # Restrict time from 5am to 8pm
    df_community = df_community[(df_community["datetime"].dt.hour >= 5) & (df_community["datetime"].dt.hour <= 20)]

    df_community.to_csv(f"./community_data/{community}.csv", index=False)
    
    #df_train_community = df_train[df_train["Customer"].isin(cid)]
    #df_test_community = df_test[df_test["Customer"].isin(cid)]
    
    #df_train_community.to_csv(f"./community_data/{community}_train.csv", index=False)
    #df_test_community.to_csv(f"./community_data/{community}_test.csv", index=False)

Processing community_1 with customer IDs: range(1, 76)
Processing community_2 with customer IDs: range(76, 151)
Processing community_3 with customer IDs: range(151, 226)
Processing community_4 with customer IDs: range(226, 301)
